# Emerging Markets Debt Strategic Allocation
## Systematic EMD Strategy with PPP Valuation & Yield Spread Signals

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import yaml
from pathlib import Path
import sys

sys.path.append('..')
from signals import EMDSignals
from risk_manager import RiskManager

sns.set_style('darkgrid')
%matplotlib inline

## 1. Load Configuration

In [ ]:
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Strategy Configuration:")
print(f"EMD Allocation: {config['strategy']['allocation_pct']:.1%}")
print(f"Local/Hard Split: {config['strategy']['local_hard_split']}")
print(f"Rebalance: {config['strategy']['rebalance_frequency']}")

## 2. Download Data

In [ ]:
# EMD ETFs
tickers = {
    'EMLC': 'VanEck EM Local Currency',  # Local currency
    'EMB': 'iShares EM USD Bonds',        # Hard currency
    'SPY': 'S&P 500',
    'AGG': 'US Aggregate Bonds',
    '^VIX': 'VIX'
}

start_date = config['backtest']['start_date']
end_date = config['backtest']['end_date']

print(f"Downloading data from {start_date} to {end_date}...")
data = yf.download(list(tickers.keys()), start=start_date, end=end_date)['Adj Close']
data = data.dropna()

print(f"\nData shape: {data.shape}")
print(f"Date range: {data.index[0]} to {data.index[-1]}")
data.head()

## 3. Calculate Returns

In [ ]:
returns = data.pct_change().dropna()

# Calculate cumulative returns
cum_returns = (1 + returns).cumprod()

print("Annualized Returns:")
for ticker in tickers.keys():
    if ticker in returns.columns:
        ann_return = returns[ticker].mean() * 252
        ann_vol = returns[ticker].std() * np.sqrt(252)
        sharpe = ann_return / ann_vol if ann_vol > 0 else 0
        print(f"{ticker:6s}: {ann_return:6.2%} | Vol: {ann_vol:6.2%} | Sharpe: {sharpe:5.2f}")

## 4. Backtest Strategy

In [ ]:
def backtest_emd_strategy(returns_df, config):
    """Backtest EMD strategic allocation"""
    
    emd_alloc = config['strategy']['allocation_pct']
    local_split, hard_split = config['strategy']['local_hard_split']
    rebalance_freq = 63  # Quarterly
    
    # Initialize portfolio
    portfolio_value = [config['backtest']['initial_capital']]
    dates = returns_df.index
    
    # Target weights
    weights = {
        'EMLC': emd_alloc * local_split,
        'EMB': emd_alloc * hard_split,
        'SPY': (1 - emd_alloc) * 0.6,
        'AGG': (1 - emd_alloc) * 0.4
    }
    
    last_rebalance = 0
    
    for i, date in enumerate(dates):
        # Check VIX trigger
        if '^VIX' in returns_df.columns:
            vix_level = data.loc[date, '^VIX'] if date in data.index else 15
            if vix_level > config['risk']['vix_reduction_trigger']:
                # Reduce EMD allocation
                emd_alloc_adj = emd_alloc * 0.5
                weights = {
                    'EMLC': emd_alloc_adj * local_split,
                    'EMB': emd_alloc_adj * hard_split,
                    'SPY': (1 - emd_alloc_adj) * 0.6,
                    'AGG': (1 - emd_alloc_adj) * 0.4
                }
        
        # Calculate daily return
        daily_return = sum(weights.get(ticker, 0) * returns_df.loc[date, ticker] 
                          for ticker in weights.keys() if ticker in returns_df.columns)
        
        # Apply transaction costs on rebalance days
        if i - last_rebalance >= rebalance_freq:
            tc_local = config['costs']['local_currency_bps'] / 10000
            tc_hard = config['costs']['hard_currency_bps'] / 10000
            tc_equity = 5 / 10000  # 5 bps for equities
            
            total_tc = (weights['EMLC'] * tc_local + 
                       weights['EMB'] * tc_hard + 
                       weights['SPY'] * tc_equity + 
                       weights['AGG'] * tc_equity)
            
            daily_return -= total_tc
            last_rebalance = i
        
        # Update portfolio value
        new_value = portfolio_value[-1] * (1 + daily_return)
        portfolio_value.append(new_value)
    
    portfolio_value = portfolio_value[1:]  # Remove initial value
    portfolio_returns = pd.Series(portfolio_value, index=dates).pct_change().dropna()
    
    return portfolio_returns, pd.Series(portfolio_value, index=dates)

# Run backtest
strategy_returns, strategy_value = backtest_emd_strategy(returns, config)

# Benchmark: 60/40
benchmark_returns = 0.6 * returns['SPY'] + 0.4 * returns['AGG']
benchmark_value = config['backtest']['initial_capital'] * (1 + benchmark_returns).cumprod()

print("Backtest completed successfully!")

## 5. Performance Metrics

In [ ]:
def calculate_metrics(returns_series, name="Strategy"):
    """Calculate performance metrics"""
    total_return = (1 + returns_series).prod() - 1
    ann_return = returns_series.mean() * 252
    ann_vol = returns_series.std() * np.sqrt(252)
    sharpe = ann_return / ann_vol if ann_vol > 0 else 0
    
    # Max drawdown
    cum_returns = (1 + returns_series).cumprod()
    running_max = cum_returns.expanding().max()
    drawdown = (cum_returns - running_max) / running_max
    max_dd = drawdown.min()
    
    # Sortino ratio
    downside_returns = returns_series[returns_series < 0]
    downside_vol = downside_returns.std() * np.sqrt(252)
    sortino = ann_return / downside_vol if downside_vol > 0 else 0
    
    metrics = {
        'Total Return': total_return,
        'Annual Return': ann_return,
        'Annual Volatility': ann_vol,
        'Sharpe Ratio': sharpe,
        'Sortino Ratio': sortino,
        'Max Drawdown': max_dd
    }
    
    return pd.Series(metrics, name=name)

# Calculate metrics
strategy_metrics = calculate_metrics(strategy_returns, "EMD Strategy")
benchmark_metrics = calculate_metrics(benchmark_returns, "60/40 Benchmark")

comparison = pd.DataFrame([strategy_metrics, benchmark_metrics]).T
comparison['Difference'] = comparison['EMD Strategy'] - comparison['60/40 Benchmark']

print("\n" + "="*70)
print("PERFORMANCE COMPARISON")
print("="*70)
print(comparison.to_string())
print("="*70)

## 6. Visualization

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Cumulative returns
axes[0].plot(strategy_value.index, strategy_value.values, label='EMD Strategy', linewidth=2)
axes[0].plot(benchmark_value.index, benchmark_value.values, label='60/40 Benchmark', linewidth=2, alpha=0.7)
axes[0].set_title('Cumulative Portfolio Value', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Portfolio Value ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Drawdown
strategy_cum = (1 + strategy_returns).cumprod()
strategy_dd = (strategy_cum - strategy_cum.expanding().max()) / strategy_cum.expanding().max()
benchmark_cum = (1 + benchmark_returns).cumprod()
benchmark_dd = (benchmark_cum - benchmark_cum.expanding().max()) / benchmark_cum.expanding().max()

axes[1].fill_between(strategy_dd.index, strategy_dd.values, 0, alpha=0.3, label='EMD Strategy')
axes[1].fill_between(benchmark_dd.index, benchmark_dd.values, 0, alpha=0.3, label='60/40 Benchmark')
axes[1].set_title('Drawdown', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Drawdown')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Rolling correlation
rolling_corr = strategy_returns.rolling(90).corr(benchmark_returns)
axes[2].plot(rolling_corr.index, rolling_corr.values, linewidth=2, color='purple')
axes[2].axhline(y=0.6, color='r', linestyle='--', label='Alert Threshold (0.6)')
axes[2].set_title('90-Day Rolling Correlation (Strategy vs Benchmark)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Correlation')
axes[2].set_xlabel('Date')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../emd_strategy_performance.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. In-Sample vs Out-of-Sample Analysis

In [ ]:
is_end = config['backtest']['in_sample_end']

# Split data
is_strategy = strategy_returns[:is_end]
oos_strategy = strategy_returns[is_end:]
is_benchmark = benchmark_returns[:is_end]
oos_benchmark = benchmark_returns[is_end:]

# Calculate metrics for each period
is_strategy_metrics = calculate_metrics(is_strategy, "IS Strategy")
oos_strategy_metrics = calculate_metrics(oos_strategy, "OOS Strategy")
is_benchmark_metrics = calculate_metrics(is_benchmark, "IS Benchmark")
oos_benchmark_metrics = calculate_metrics(oos_benchmark, "OOS Benchmark")

period_comparison = pd.DataFrame([
    is_strategy_metrics, oos_strategy_metrics,
    is_benchmark_metrics, oos_benchmark_metrics
]).T

print("\n" + "="*90)
print("IN-SAMPLE (2003-2012) vs OUT-OF-SAMPLE (2013-2023) ANALYSIS")
print("="*90)
print(period_comparison.to_string())
print("="*90)

# Check if strategy meets targets
sharpe_improvement = oos_strategy_metrics['Sharpe Ratio'] - oos_benchmark_metrics['Sharpe Ratio']
dd_increase = oos_strategy_metrics['Max Drawdown'] - oos_benchmark_metrics['Max Drawdown']

print(f"\nOOS Sharpe Improvement: {sharpe_improvement:.3f} (Target: >0.15)")
print(f"OOS Max DD Increase: {dd_increase:.3%} (Target: <2%)")

if sharpe_improvement > 0.15 and dd_increase < 0.02:
    print("\n✓ Strategy MEETS performance targets!")
else:
    print("\n✗ Strategy does NOT meet performance targets.")

## 8. Risk Analysis

In [ ]:
# Initialize risk manager
risk_mgr = RiskManager(config)

# Stress test scenarios
final_value = strategy_value.iloc[-1]

print("\n" + "="*70)
print("STRESS TEST RESULTS")
print("="*70)

for scenario in ['taper_tantrum', 'covid_crash']:
    result = risk_mgr.apply_stress_test(final_value, scenario)
    print(f"\n{scenario.upper().replace('_', ' ')}:")
    print(f"  Portfolio Value: ${result['portfolio_value']:,.2f}")
    print(f"  Loss: {result['loss_pct']:.2%}")

print("\n" + "="*70)

## 9. Export Results

In [ ]:
# Save results
results_df = pd.DataFrame({
    'Strategy_Value': strategy_value,
    'Benchmark_Value': benchmark_value,
    'Strategy_Returns': strategy_returns,
    'Benchmark_Returns': benchmark_returns
})

results_df.to_csv('../backtest_results.csv')
comparison.to_csv('../performance_metrics.csv')

print("Results exported to:")
print("  - backtest_results.csv")
print("  - performance_metrics.csv")
print("  - emd_strategy_performance.png")